# 2th_hometask

Домашка № 2:

Обработать данные, по аналогии, что в первой домашке, но добавляем подготовку и анализ категориальных значений.

Будет круто, если вы покажете, что умеете применять label encoding и one-hot.

К моделям не привязываемся.


Пояснения к столбцам:

People

- ID: Customer's unique identifier
- Year_Birth: Customer's birth year
- Education: Customer's education level
- Marital_Status: Customer's marital status
- Income: Customer's yearly household income
- Kidhome: Number of children in customer's household
- Teenhome: Number of teenagers in customer's household
- Dt_Customer: Date of customer's enrollment with the company
- Recency: Number of days since customer's last purchase
- Complain: 1 if customer complained in the last 2 years, 0 otherwise

Products

- MntWines: Amount spent on wine in last 2 years
- MntFruits: Amount spent on fruits in last 2 years
- MntMeatProducts: Amount spent on meat in last 2 years
- MntFishProducts: Amount spent on fish in last 2 years
- MntSweetProducts: Amount spent on sweets in last 2 years
- MntGoldProds: Amount spent on gold in last 2 years

Promotion

- NumDealsPurchases: Number of purchases made with a discount
- AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise
- AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise
- AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise
- AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise
- AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise
- Response: 1 if customer accepted the offer in the last campaign, 0 otherwise

Place

- NumWebPurchases: Number of purchases made through the company’s web site
- NumCatalogPurchases: Number of purchases made using a catalogue
- NumStorePurchases: Number of purchases made directly in stores
- NumWebVisitsMonth: Number of visits to company’s web site in the last month

### Import Section

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder

### Function "Reduce Memory Usage"

In [2]:
def reduce_memory_usage(df):
    
    # СДЕЛАТЬ ПРОВЕРКУ type(df) == pd.DataFrame
    
    initial_memory_usage = df.memory_usage().sum() / 1024 / 1024
    print(f'Initial memory usage of dataframe:\t{initial_memory_usage:.3} Mb')
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            # Можно было бы сделать оптимизацию вплоть до типа "float16", однако данный тип данных,
            # как сообщается от сообщества аналитиков данных, плохо поддерживается некоторыми библиотеками
            
            if str(col_type)[:5] == 'float':
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float64).min and c_max < np.finfo(np.float64).max:
                    df[col] = df[col].astype(np.float64)
        
        else:
            df[col] = df[col].astype('category')
    
    final_memory_usage = df.memory_usage().sum() / 1024 / 1024
    print(f'Final memory usage of dataframe:\t{final_memory_usage:.3} Mb')
    
    comparison = np.round(100 * (initial_memory_usage - final_memory_usage) / initial_memory_usage, 3)
    print(f'Memory usage has been decreased by:\t{comparison} %')
    
    return df

### Path Section

In [3]:
PATH_DATA = r'marketing_campaign.csv'
PATH_DATA_PREPROCESSED = r'marketing_campaign_preprocessed.csv'

## Exploratory Data Analysis

In [4]:
df_data = pd.read_csv(PATH_DATA, sep=f'\t')

In [5]:
df_data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635.0,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11.0,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426.0,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11.0,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173.0,...,5,0,0,0,0,0,0,3,11,0


In [6]:
df_data.describe()

,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
count,14240.000000,14240.000000,14061.000000,14240.000000,14240.000000,14240.000000,13006.000000,13006.000000,14240.000000,14240.000000,...,14240.000000,14240.000000,14240.000000,14240.000000,14240.000000,14240.000000,14240.000000,14240.0,14240.0,14240.000000
mean,5567.952809,1968.708006,52243.288529,0.445365,0.508848,49.355758,301.440412,25.996924,168.189115,37.646138,...,5.319171,0.073174,0.071419,0.072683,0.066292,0.012500,0.009902,3.0,11.0,0.145927
std,3256.562439,12.010372,24861.756010,0.536045,0.542387,28.781473,332.435860,39.347185,227.955620,54.454956,...,2.433184,0.260431,0.257532,0.259624,0.248801,0.111106,0.099017,0.0,0.0,0.353045
min,0.000000,1893.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
25%,2794.500000,1959.000000,35322.000000,0.000000,0.000000,24.000000,24.000000,1.000000,16.000000,3.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
50%,5492.000000,1970.000000,51390.000000,0.000000,0.000000,50.000000,174.000000,8.000000,68.000000,12.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
75%,8420.000000,1977.000000,68148.000000,1.000000,1.000000,74.000000,496.000000,33.000000,228.000000,50.000000,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
max,11191.000000,1996.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,...,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.0,11.0,1.000000


In [7]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14240 entries, 0 to 14239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   14240 non-null  int64  
 1   Year_Birth           14240 non-null  int64  
 2   Education            14240 non-null  object 
 3   Marital_Status       14240 non-null  object 
 4   Income               14061 non-null  float64
 5   Kidhome              14240 non-null  int64  
 6   Teenhome             14240 non-null  int64  
 7   Dt_Customer          14240 non-null  object 
 8   Recency              14240 non-null  int64  
 9   MntWines             13006 non-null  float64
 10  MntFruits            13006 non-null  float64
 11  MntMeatProducts      14240 non-null  int64  
 12  MntFishProducts      14240 non-null  int64  
 13  MntSweetProducts     14240 non-null  int64  
 14  MntGoldProds         14240 non-null  int64  
 15  NumDealsPurchases    14240 non-null 

In [8]:
df_null = pd.DataFrame(df_data.isna().sum(), columns=['Nulls'])
df_null = df_null[df_null['Nulls'] > 0]
df_null['Total'] = df_data.shape[0]
df_null['Percentage'] = df_null['Nulls'] / df_null['Total'] * 100
df_null

,Nulls,Total,Percentage
Income,179,14240,1.257022
MntWines,1234,14240,8.665730
MntFruits,1234,14240,8.665730


##### Признаки Income, MntWines, MntFruits содержат пропуски и должны быть обработаны.
##### Признаки Education, Marital_Status могут быть переведены в категориальные.
##### Также необходимо добавить новый признак, для обработанных записей.

## Data Preprocessing

In [9]:
df_data_preprocessed = df_data.drop_duplicates().copy()

##### Поскольку признак ID уникален для каждого клиента, то будем использовать его в качестве индекса. Для прогнозирования он не пригодится.

In [10]:
df_data_preprocessed.index = df_data_preprocessed['ID']
df_data_preprocessed = df_data_preprocessed.drop(columns=['ID']).copy()
df_data_preprocessed.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
ID,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635.0,88.0,...,7,0,0,0,0,0,0,3,11,1
2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11.0,1.0,...,5,0,0,0,0,0,0,3,11,0
4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426.0,49.0,...,4,0,0,0,0,0,0,3,11,0
6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11.0,4.0,...,6,0,0,0,0,0,0,3,11,0
5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173.0,43.0,...,5,0,0,0,0,0,0,3,11,0


##### Признаки MntWines и MntFruits могут быть заполнены нулями, так как отсутствие информации о тратах в данных категория можно интерпретировать как отсутствие трат.
##### Записи с отсутствующими значениям признака Income следует исключить, так как их доля мала ~1,25%, влияние признака на целевую переменную велико, а качественное заполнение пропусков крайне тяжело для каждого конкретного клиента и может принести больше вреда, чем пользы.

In [11]:
df_data_preprocessed['preprocessed'] = 0

df_data_preprocessed = df_data_preprocessed[~df_data_preprocessed['Income'].isna()].copy()

df_data_preprocessed['preprocessed'][df_data_preprocessed['MntWines'].isna()] = 1
df_data_preprocessed['MntWines'][df_data_preprocessed['MntWines'].isna()] = 0

df_data_preprocessed['preprocessed'][df_data_preprocessed['MntFruits'].isna()] = 1
df_data_preprocessed['MntFruits'][df_data_preprocessed['MntFruits'].isna()] = 1

df_data_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4108 entries, 5524 to 7451
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year_Birth           4108 non-null   int64  
 1   Education            4108 non-null   object 
 2   Marital_Status       4108 non-null   object 
 3   Income               4108 non-null   float64
 4   Kidhome              4108 non-null   int64  
 5   Teenhome             4108 non-null   int64  
 6   Dt_Customer          4108 non-null   object 
 7   Recency              4108 non-null   int64  
 8   MntWines             4108 non-null   float64
 9   MntFruits            4108 non-null   float64
 10  MntMeatProducts      4108 non-null   int64  
 11  MntFishProducts      4108 non-null   int64  
 12  MntSweetProducts     4108 non-null   int64  
 13  MntGoldProds         4108 non-null   int64  
 14  NumDealsPurchases    4108 non-null   int64  
 15  NumWebPurchases      4108 non-null 

C:\Users\SERGEY~1\AppData\Local\Temp/ipykernel_14240/1641502974.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_preprocessed['preprocessed'][df_data_preprocessed['MntWines'].isna()] = 1
C:\Users\SERGEY~1\AppData\Local\Temp/ipykernel_14240/1641502974.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_preprocessed['MntWines'][df_data_preprocessed['MntWines'].isna()] = 0
C:\Users\SERGEY~1\AppData\Local\Temp/ipykernel_14240/1641502974.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

##### Dt_Customer - дата регистрации пользователя. В теории данный признак может быть использован для вычисления коэффициентов активности пользователя: [(текущая дата) - (дата регистрации)] / (общая сумма трат). Однако оценка полезности данного преобразования признака в настоящий момент слишком затратна. Также признак можно было было разбить на три числовых признака: год, месяц, день, что однако также не гарантирует полезности данной информации. Искючим признак в настоящий момент, оставив его потенциал для усовершенствования модели в дальнейшем.

In [12]:
df_data_preprocessed = df_data_preprocessed.drop(columns=['Dt_Customer'])

##### Education - нельзя однозначно сказать насколько одно образование лучше другого, поэтому применим OneHotEncoder для преобразования признака.

In [13]:
ohe_Education = OneHotEncoder()

df_Education = ohe_Education.fit_transform(df_data_preprocessed[['Education']])
df_Education = pd.DataFrame(df_Education.toarray(), columns=ohe_Education.categories_[0])
df_Education.index = df_data_preprocessed.index

df_data_preprocessed = df_data_preprocessed.join(df_Education)
df_data_preprocessed = df_data_preprocessed.drop(columns=['Education'])
df_data_preprocessed = df_data_preprocessed.drop_duplicates()
df_data_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3867 entries, 0 to 11191
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year_Birth           3867 non-null   int64  
 1   Marital_Status       3867 non-null   object 
 2   Income               3867 non-null   float64
 3   Kidhome              3867 non-null   int64  
 4   Teenhome             3867 non-null   int64  
 5   Recency              3867 non-null   int64  
 6   MntWines             3867 non-null   float64
 7   MntFruits            3867 non-null   float64
 8   MntMeatProducts      3867 non-null   int64  
 9   MntFishProducts      3867 non-null   int64  
 10  MntSweetProducts     3867 non-null   int64  
 11  MntGoldProds         3867 non-null   int64  
 12  NumDealsPurchases    3867 non-null   int64  
 13  NumWebPurchases      3867 non-null   int64  
 14  NumCatalogPurchases  3867 non-null   int64  
 15  NumStorePurchases    3867 non-null   

##### Аналогичное преобразование проводим для признака Marital_Status.

In [14]:
ohe_Marital_Status = OneHotEncoder()

df_Marital_Status = ohe_Marital_Status.fit_transform(df_data_preprocessed[['Marital_Status']])
df_Marital_Status = pd.DataFrame(df_Marital_Status.toarray(), columns=ohe_Marital_Status.categories_[0])
df_Marital_Status.index = df_data_preprocessed.index

df_data_preprocessed = df_data_preprocessed.join(df_Marital_Status)
df_data_preprocessed = df_data_preprocessed.drop(columns=['Marital_Status'])
df_data_preprocessed = df_data_preprocessed.drop_duplicates()
df_data_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3867 entries, 0 to 11191
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year_Birth           3867 non-null   int64  
 1   Income               3867 non-null   float64
 2   Kidhome              3867 non-null   int64  
 3   Teenhome             3867 non-null   int64  
 4   Recency              3867 non-null   int64  
 5   MntWines             3867 non-null   float64
 6   MntFruits            3867 non-null   float64
 7   MntMeatProducts      3867 non-null   int64  
 8   MntFishProducts      3867 non-null   int64  
 9   MntSweetProducts     3867 non-null   int64  
 10  MntGoldProds         3867 non-null   int64  
 11  NumDealsPurchases    3867 non-null   int64  
 12  NumWebPurchases      3867 non-null   int64  
 13  NumCatalogPurchases  3867 non-null   int64  
 14  NumStorePurchases    3867 non-null   int64  
 15  NumWebVisitsMonth    3867 non-null   

In [15]:
df_data_preprocessed = reduce_memory_usage(df_data_preprocessed)

Initial memory usage of dataframe:	1.18 Mb
Final memory usage of dataframe:	0.369 Mb
Memory usage has been decreased by:	68.75 %


In [16]:
df_data_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3867 entries, 0 to 11191
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year_Birth           3867 non-null   int16  
 1   Income               3867 non-null   float32
 2   Kidhome              3867 non-null   int8   
 3   Teenhome             3867 non-null   int8   
 4   Recency              3867 non-null   int8   
 5   MntWines             3867 non-null   float32
 6   MntFruits            3867 non-null   float32
 7   MntMeatProducts      3867 non-null   int16  
 8   MntFishProducts      3867 non-null   int16  
 9   MntSweetProducts     3867 non-null   int16  
 10  MntGoldProds         3867 non-null   int16  
 11  NumDealsPurchases    3867 non-null   int8   
 12  NumWebPurchases      3867 non-null   int8   
 13  NumCatalogPurchases  3867 non-null   int8   
 14  NumStorePurchases    3867 non-null   int8   
 15  NumWebVisitsMonth    3867 non-null   

In [17]:
df_data_preprocessed.to_csv(PATH_DATA_PREPROCESSED)